In [2]:
#Dependencies
!pip install dspy-ai[chromadb] -Uqq
!pip install termcolor -Uqq
!pip install sqlalchemy -Uqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

#### Importing utilities

In [3]:
#imports
from bs4 import BeautifulSoup
import urllib.request
import ssl
import openai
import os
import requests
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import pandas as pd
import json
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
)
import re
from sqlalchemy import inspect
import sqlalchemy
from sqlalchemy import text
import dspy
from typing import List
from termcolor import colored
import chromadb

In [4]:
from dsp.modules.cache_utils import cache_turn_on

cache_turn_on

True

### Loading data

In [11]:
df = pd.read_csv("ERP_Data/AP_SUPPLIERS.csv")

In [12]:
df.head()

,VENDOR_ID,LAST_UPDATE_DATE,LAST_UPDATED_BY,VENDOR_NAME,VENDOR_NAME_ALT,SEGMENT1,SUMMARY_FLAG,ENABLED_FLAG,LAST_UPDATE_LOGIN,CREATION_DATE,...,AMOUNT_INCLUDES_TAX_FLAG,PROGRAM_UPDATE_DATE,VAT_REGISTRATION_NUM,AUTO_CALCULATE_INTEREST_FLAG,EXCLUDE_FREIGHT_FROM_DISCOUNT,MATCH_OPTION,OFFSET_TAX_FLAG,PARTY_ID,TCA_SYNC_VENDOR_NAME,TCA_SYNC_VAT_REG_NUM
0,7136,5/25/2024,Sylvia Brooks,Ware-Williams,"Hernandez, George and Ramirez",throw,Y,N,649559,2/28/2024,...,N,4/10/2024,232818,N,N,foot,Y,962003,century,637630
1,8667,3/29/2024,Samuel Cunningham,Burnett PLC,"Larson, Mcdaniel and Jarvis",trouble,Y,Y,887153,3/5/2024,...,Y,5/6/2024,983779,Y,Y,commercial,Y,936681,like,811016
2,2023,2/24/2024,Monica Scott,Martin-Hudson,"Stewart, Franklin and George",say,Y,N,435433,6/13/2024,...,Y,6/12/2024,995433,N,N,outside,N,644458,lawyer,959161
3,6187,3/24/2024,Mark Reynolds,Spencer-Garner,Sullivan Ltd,when,N,Y,997477,6/16/2024,...,Y,3/16/2024,776490,Y,Y,father,Y,208584,truth,800789
4,9934,4/21/2024,Heidi Simpson,Moore-Gill,"Glenn, Dawson and Lewis",if,Y,N,600852,2/2/2024,...,Y,5/21/2024,520238,Y,N,without,N,268863,imagine,881960


## BUILD TABLE NAMES AND METADATA

* Here I use a DSPy signature given the first 10 rows of the dataframe, I generate the table name and table explanation. It will help to dynamically select the correct table based on the query.

In [13]:
df.head(10).to_csv()

',VENDOR_ID,LAST_UPDATE_DATE,LAST_UPDATED_BY,VENDOR_NAME,VENDOR_NAME_ALT,SEGMENT1,SUMMARY_FLAG,ENABLED_FLAG,LAST_UPDATE_LOGIN,CREATION_DATE,CREATED_BY,EMPLOYEE_ID,VENDOR_TYPE_LOOKUP_CODE,ONE_TIME_FLAG,TERMS_ID,ALWAYS_TAKE_DISC_FLAG,PAY_DATE_BASIS_LOOKUP_CODE,PAYMENT_PRIORITY,INVOICE_CURRENCY_CODE,PAYMENT_CURRENCY_CODE,INVOICE_AMOUNT_LIMIT,HOLD_ALL_PAYMENTS_FLAG,HOLD_FUTURE_PAYMENTS_FLAG,START_DATE_ACTIVE,END_DATE_ACTIVE,PAYMENT_METHOD_LOOKUP_CODE,BANK_ACCOUNT_NAME,BANK_ACCOUNT_NUM,BANK_NUM,BANK_ACCOUNT_TYPE,WOMEN_OWNED_FLAG,SMALL_BUSINESS_FLAG,HOLD_FLAG,TERMS_DATE_BASIS,HOLD_UNMATCHED_INVOICES_FLAG,AP_TAX_ROUNDING_RULE,AUTO_TAX_CALC_FLAG,AMOUNT_INCLUDES_TAX_FLAG,PROGRAM_UPDATE_DATE,VAT_REGISTRATION_NUM,AUTO_CALCULATE_INTEREST_FLAG,EXCLUDE_FREIGHT_FROM_DISCOUNT,MATCH_OPTION,OFFSET_TAX_FLAG,PARTY_ID,TCA_SYNC_VENDOR_NAME,TCA_SYNC_VAT_REG_NUM\n0,7136,5/25/2024,Sylvia Brooks,Ware-Williams,"Hernandez, George and Ramirez",throw,Y,N,649559,2/28/2024,Shelly Glover,3554,reason,N,7524,Y,result,10

In [18]:
import os
key = "sk-shay-dev-nFHOym7NFDr95khZ87GYT3BlbkFJrqNYHwLMJFRDmsruwxUJ"
os.environ["OPENAI_API_KEY"] = key
openai.api_key = key

In [6]:
import dspy
# turbo = dspy.OpenAI(model='gpt-3.5-turbo-instruct', max_tokens=250)
turbo = dspy.OpenAI(model='gpt-3.5-turbo', max_tokens=250)
dspy.settings.configure(lm=turbo)

class SQLTableMetadata(dspy.Signature):
    """Give a suitable table name and description about the given table"""
    pandas_dataframe_str = dspy.InputField(desc="First 10 rows of a pandas dataframe delimited by newline character")
    table_name = dspy.OutputField(desc="suitable table name")
    table_summary = dspy.OutputField(desc="a summary about the table")

class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(SQLTableMetadata)

    def forward(self, pandas_dataframe_str):
        return self.prog(pandas_dataframe_str=pandas_dataframe_str)

cot = CoT()

In [15]:
cot(pandas_dataframe_str = df.head(10).to_csv())

Prediction(
    rationale='produce the table summary. We have a dataset containing information about different vendors, including their IDs, names, payment details, currency codes, payment methods, and various flags indicating specific attributes. This table provides a comprehensive overview of vendor information that can be used for analysis and decision-making.',
    table_name='Vendor Information Table',
    table_summary='This table contains detailed information about various vendors, including their IDs, names, payment details, currency codes, payment methods, and flags indicating specific attributes. It can be used for analysis and decision-making related to vendor management.'
)

In [7]:
import pandas as pd
import os
import json
from tqdm import tqdm

def sanitize_column_name(col_name):
    # Remove special characters and replace spaces with underscores
    return re.sub(r"\W+", "_", col_name)

# Specify the directory containing the processed data
processed_dir = "ERP_Data"

# Initialize an empty list to store dictionaries
all_infos_dict = []

# Iterate over all directories within the processed directory
# for country in os.listdir(processed_dir):
#     print(country)
#     country_dir = os.path.join(processed_dir, country)
#     os.makedirs(country_dir, exist_ok=True)
final_dir = os.path.join(processed_dir, "")


# Iterate over all CSV files within the country directory
for csv_file in tqdm(os.listdir(processed_dir)):
    full_file_name = os.path.join(final_dir, csv_file)
    if not full_file_name.endswith(".csv"):
        continue
    # print(full_file_name)

    # Load the CSV file into a DataFrame
    df = pd.read_csv(full_file_name)
    df.dropna(how='all', inplace=True)

    # Convert the DataFrame to a dictionary (example)
    info_dict = df.to_dict(orient='records')[0]  # Convert the first row to a dict

    # Save the DataFrame to a CSV file
    save_name = full_file_name.split(".")[0].split(os.sep)[-1]
    save_file_path = os.path.join(final_dir, f"{save_name}_processed.csv")
    df.to_csv(save_file_path, index=False)

    # Save the info_dict to a JSON file
    with open(save_file_path.replace('.csv', '.json'), 'w') as outfile:
        json.dump(info_dict, outfile)

print("Processing completed.")


100%|██████████| 7/7 [00:00<00:00, 32.82it/s]

Processing completed.


In [8]:
processed_dir = "ERP_Data"
dfs_str = []
for files in tqdm(os.listdir(processed_dir)): # Building the summary and name for tables
    if files.endswith("processed.csv"):
        file_name = files.split(".")[0]
        csv_file_path = os.path.join(processed_dir,files)
        df = pd.read_csv(csv_file_path,index_col=False)
        json_file_path = os.path.join(processed_dir,f"{file_name}.json")
        print(f"\n#### {json_file_path}")
        with open(json_file_path,'r') as f:
            data = json.loads(f.read())
            if 'table_name' in data and 'table_summary' in data:
                # if data['table_name'] == "" or data['table_summary'] == "":
                if data['table_summary'] == "":
                    pass
                else:
                    continue
        dfs_str.append(df.head(10).to_csv())
        table_preds = cot(pandas_dataframe_str = df.head(10).to_csv())
        print(table_preds.table_name)
        print(table_preds.table_summary)
        data['table_name'] = file_name
        data['table_summary'] = table_preds.table_summary
        with open(json_file_path,'w') as f:
            json.dump(data, f)

  0%|          | 0/21 [00:00<?, ?it/s]


#### ERP_Data/PO_HEADERS_ALL_processed.json


 33%|███▎      | 7/21 [00:01<00:02,  5.59it/s]

Purchase_Order_Details
This table contains detailed information about purchase orders, including header ID, agent ID, type, last update date, vendor details, shipping information, currency, approval status, and more.

#### ERP_Data/AP_CHECKS_ALL_processed.json


 38%|███▊      | 8/21 [00:02<00:05,  2.47it/s]

...

Accounting Events Data
This table contains detailed information about various accounting events, including event ID, date, amount, payment details, bank account information, and more. Each row represents a different accounting event.

#### ERP_Data/PO_DISTRIBUTIONS_ALL_processed.json


 76%|███████▌  | 16/21 [00:09<00:03,  1.50it/s]

Purchase_Order_Distributions
This table contains details of purchase order distributions, including distribution ID, dates, quantities, amounts, and flags related to encumbrance and invoicing.

#### ERP_Data/AP_INVOICES_ALL_processed.json


100%|██████████| 21/21 [00:11<00:00,  1.83it/s]

Invoice_Details
This table contains detailed information about various invoices, including invoice ID, invoice number, vendor details, invoice amount, payment details, approval status, and more.


## NEXT TASKS
1. Build database for each table
2. Embed the table summary and table SCHEMA. Also, embed the table rows
3. Retrieval at table level and embed the rows to retrieve relevant rows from the retrieved schema of table
4. Text-to-SQL pipeline

#### BUILD THE SQLITE DATABASE FROM THE CSV FILES

In [9]:
# Function to create a sanitized column name
def sanitize_column_name(col_name):
    # Remove special characters and replace spaces with underscores
    return re.sub(r"\W+", "_", col_name)


# Function to create a table from a DataFrame using SQLAlchemy
def create_table_from_dataframe(
    df: pd.DataFrame, table_name: str, engine, metadata_obj
):
    # Sanitize column names
    sanitized_columns = {col: sanitize_column_name(col) for col in df.columns}
    df = df.rename(columns=sanitized_columns)

    # Dynamically create columns based on DataFrame columns and data types
    columns = [
        Column(col, String if dtype == "object" else Integer)
        for col, dtype in zip(df.columns, df.dtypes)
    ]

    # Create a table with the defined columns
    table = Table(table_name, metadata_obj, *columns)

    # Create the table in the database
    metadata_obj.create_all(engine)

    # Insert data from DataFrame into the table
    with engine.connect() as conn:
        for _, row in df.iterrows():
            insert_stmt = table.insert().values(**row.to_dict())
            conn.execute(insert_stmt)
        conn.commit()

## DATABASE CREATION

In [10]:
processed_dir  = "ERP_Data"
def sqlalchemy_engine(region:str):
    """Create a SQLAlchemy engine for the given region"""
    # Create a SQLAlchemy database for each region
    engine = create_engine(f"sqlite:///{region}.db")
    metadata_obj = MetaData()
    # region_path = os.path.join(processed_dir,region)
    dfs = []
    for dataframes_path in os.listdir(processed_dir):
        if dataframes_path.endswith("processed.csv"):
            df = pd.read_csv(os.path.join(processed_dir,dataframes_path),index_col=False)
            dfs.append((dataframes_path,df))
    pbar = tqdm(total=len(dfs),desc=f"Creating tables for {region}")
    for _, df_table_name in enumerate(dfs):
        table_name = df_table_name[0]
        table_name = table_name.split(".")[0]
        df = df_table_name[1]
        # print(f"Creating table: {table_name}")
        create_table_from_dataframe(df,table_name, engine, metadata_obj)
        # print(f"Done creating table for: {table_name}")
        pbar.update(1)
    return engine

In [11]:
db_engine = sqlalchemy_engine("main")

Creating tables for main: 100%|██████████| 7/7 [00:04<00:00,  1.45it/s]


In [12]:
def get_table_infos(sql_engine:sqlalchemy.engine.base.Engine,region:str):
    """Get all the tables info in the database based on the given region"""
    inspector = inspect(sql_engine)
    table_names = inspector.get_table_names()
    table_infos_dict = {tb: [] for tb in table_names}
    for tb in table_names:
        column_dict = inspector.get_columns(tb)
        schema_str = ""
        primary_keys = []
        for col in column_dict:
            schema_str += f"{col['name']} ({col['type']}), "
            if col["primary_key"] not in primary_keys:
                primary_keys.append(col["name"])
        with open(os.path.join(processed_dir,f"{tb}.json")) as f:
            table_info = json.loads(f.read())
        table_infos_dict[tb] = [
            {
                "table_info": f"Table {tb} has columns: {schema_str[:-2]}",
                "table_summary": f'{table_info["table_summary"]}. ',
            }
        ]
    return table_infos_dict

In [13]:
tb_dict = get_table_infos(db_engine,"main")

In [14]:
tb_dict

{'AP_CHECKS_ALL_processed': [{'table_info': 'Table AP_CHECKS_ALL_processed has columns: AMOUNT (INTEGER), BANK_ACCOUNT_ID (INTEGER), BANK_ACCOUNT_NAME (VARCHAR), CHECK_DATE (VARCHAR), CHECK_ID (INTEGER), CHECK_NUMBER (INTEGER), CURRENCY_CODE (VARCHAR), LAST_UPDATED_BY (VARCHAR), LAST_UPDATE_DATE (VARCHAR), PAYMENT_METHOD_LOOKUP_CODE (VARCHAR), PAYMENT_TYPE_FLAG (VARCHAR), ADDRESS_LINE1 (VARCHAR), CHECKRUN_NAME (VARCHAR), CITY (VARCHAR), COUNTRY (VARCHAR), CREATED_BY (VARCHAR), CREATION_DATE (VARCHAR), STATUS_LOOKUP_CODE (VARCHAR), VENDOR_NAME (VARCHAR), ZIP (INTEGER), BANK_ACCOUNT_NUM (INTEGER), BANK_ACCOUNT_TYPE (VARCHAR), BANK_NUM (INTEGER), CHECK_VOUCHER_NUM (INTEGER), CLEARED_AMOUNT (INTEGER), CLEARED_DATE (VARCHAR), ORG_ID (INTEGER), VENDOR_ID (INTEGER), VENDOR_SITE_ID (INTEGER), EXCHANGE_DATE (VARCHAR), CHECKRUN_ID (INTEGER), REQUEST_ID (INTEGER), POSITIVE_PAY_STATUS_CODE (VARCHAR), ANTICIPATED_VALUE_DATE (VARCHAR), CE_BANK_ACCT_USE_ID (INTEGER), PAYMENT_METHOD_CODE (VARCHAR), PA

In [37]:
tb_dict

{'AP_CHECKS_ALL_processed': [{'table_info': 'Table AP_CHECKS_ALL_processed has columns: AMOUNT (INTEGER), BANK_ACCOUNT_ID (INTEGER), BANK_ACCOUNT_NAME (VARCHAR), CHECK_DATE (VARCHAR), CHECK_ID (INTEGER), CHECK_NUMBER (INTEGER), CURRENCY_CODE (VARCHAR), LAST_UPDATED_BY (VARCHAR), LAST_UPDATE_DATE (VARCHAR), PAYMENT_METHOD_LOOKUP_CODE (VARCHAR), PAYMENT_TYPE_FLAG (VARCHAR), ADDRESS_LINE1 (VARCHAR), CHECKRUN_NAME (VARCHAR), CITY (VARCHAR), COUNTRY (VARCHAR), CREATED_BY (VARCHAR), CREATION_DATE (VARCHAR), STATUS_LOOKUP_CODE (VARCHAR), VENDOR_NAME (VARCHAR), ZIP (INTEGER), BANK_ACCOUNT_NUM (INTEGER), BANK_ACCOUNT_TYPE (VARCHAR), BANK_NUM (INTEGER), CHECK_VOUCHER_NUM (INTEGER), CLEARED_AMOUNT (INTEGER), CLEARED_DATE (VARCHAR), ORG_ID (INTEGER), VENDOR_ID (INTEGER), VENDOR_SITE_ID (INTEGER), EXCHANGE_DATE (VARCHAR), CHECKRUN_ID (INTEGER), REQUEST_ID (INTEGER), POSITIVE_PAY_STATUS_CODE (VARCHAR), ANTICIPATED_VALUE_DATE (VARCHAR), CE_BANK_ACCT_USE_ID (INTEGER), PAYMENT_METHOD_CODE (VARCHAR), PA

In [15]:
type(tb_dict)

dict

## EMBEDDINGS

1. Embed the table summary and table SCHEMA to get the table that the user is looking for
2. Embed the table rows for each table, so as to get relevant rows from the retrieved table

## EMBED THE TABLE SUMMARY AND TABLE SCHEMA

In [19]:
os.environ['OPENAI_API_KEY'],

('sk-shay-dev-nFHOym7NFDr95khZ87GYT3BlbkFJrqNYHwLMJFRDmsruwxUJ',)

In [34]:
from chromadb.db.base import UniqueConstraintError  # Correct import statement

In [39]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from chromadb.utils.batch_utils import create_batches
from chromadb.config import Settings

emb_fn = embedding_functions.OpenAIEmbeddingFunction(
                api_key=os.environ['OPENAI_API_KEY'],
                model_name="text-embedding-3-small")
# EMBEDDING_MODEL = "mixedbread-ai/mxbai-embed-large-v1"
# emb_fn = embedding_functions.HuggingFaceEmbeddingFunction(model_name=EMBEDDING_MODEL,api_key=os.environ["HF_API_KEY"])
def embed_table_info(region:str,tb_dict):
    """Embed the table summary and table SCHEMA to get the table that the user is looking for"""
    settings = Settings(allow_reset=True)
    client = chromadb.PersistentClient(path="./main_TABLE", settings=settings)
    # client = chromadb.PersistentClient(path=f"{region}_TABLE")
    try:
        table_collection = client.create_collection(name="table",embedding_function=emb_fn)
    except UniqueConstraintError:
        print("Collection 'table' already exists. Skipping creation.")

    table_docs = []
    table_metadata = []


    for table_name,table_data in tb_dict.items():
        table_docs.append(table_data[0]['table_info'] + ". " + table_data[0]['table_summary'])
        table_metadata.append({"table_name":table_name,'table_metadata':table_data[0]['table_info']})
    table_ids = [f"id{i}" for i in range(len(table_docs))]
    assert len(table_docs) == len(table_metadata)
    print(len(table_docs),len(table_metadata))
    # Create a batch of data to be sent to OpenAI Embedding API
    batches = create_batches(api=client,ids=table_ids, documents=table_docs, metadatas=table_metadata)
    for batch in tqdm(batches,desc="Embedding table info"):
        table_collection.add(ids=batch[0],
                    documents=batch[3],
                    metadatas=batch[2])

embed_table_info("main",tb_dict)

7 7


Embedding table info: 100%|██████████| 1/1 [00:00<00:00,  2.78it/s]


## For some strange reason, the `create_batches` was not batching the below documents, hence I had to do it manually

In [41]:

def embed_rows(region:str,batch_size:int=24):
    client = chromadb.PersistentClient(path=f"{region}_TABLE")
    # client.delete_collection(name="rows")
    settings = Settings(allow_reset=True)
    client = chromadb.PersistentClient(path="./main_TABLE", settings=settings)
    # client = chromadb.PersistentClient(path=f"{region}_TABLE")
    try:
        rows_collection = client.create_collection(name="rows",embedding_function=emb_fn)
    except UniqueConstraintError:
        print("Collection 'rows' already exists. Skipping creation.")

    rows_docs = []
    rows_metadata = []
    region_path = os.path.join(processed_dir,"")
    for df_path in os.listdir(region_path):
        df_full_path = os.path.join(region_path,df_path)
        df = pd.read_csv(df_full_path,index_col=False)
        for idx,row in df.iterrows():
            row_str = ""
            full_rows = []
            for rv in row.values:
                if isinstance(rv,str):
                    row_str+= rv + ", "
                full_rows.append(str(rv))
                row_str = row_str.replace('"',"")
                # row_str = row_str.replace("'",'"')
            full_rows_str = ", ".join(full_rows)[:-2]
            full_rows_str = full_rows_str.replace('"',"")
            rows_docs.append(row_str[:-2])
            rows_metadata.append({"table_name":df_path.split(".")[0],"region":region,"index":idx,"full_rows":full_rows_str})
    row_ids = [f"id{i}" for i in range(len(rows_docs))]
    # print(len(rows_docs),len(rows_metadata))
    assert len(rows_docs) == len(rows_metadata) == len(row_ids)
    # return rows_docs,rows_metadata,row_ids
    for start in tqdm(range(0,len(rows_docs),batch_size),desc="Embedding rows"):
        end = min(start+batch_size,len(rows_docs))
        batch_ids = row_ids[start:end]
        batch_rows = rows_docs[start:end]
        batch_metadatas = rows_metadata[start:end]
        rows_collection.add(ids=batch_ids,
                    documents=batch_rows,
                    metadatas=batch_metadatas)
    # return batches

In [42]:
region = "main"
# embed_table_info(region,tb_dict)
embed_rows(region,2000)

Embedding rows: 100%|██████████| 4/4 [00:26<00:00,  6.56s/it]


## TEXT-TO-SQL PIPELINE

### LOAD DATABASE

In [43]:
db_dict = {
    "main":db_engine,
}

def get_collections_db(region:str):
    # Get the database for the given region, table collection and row collection
    client = chromadb.PersistentClient(path=f"{region}_TABLE")
    table_collection = client.get_collection(name="table",embedding_function=emb_fn)
    row_collection = client.get_collection(name="rows",embedding_function=emb_fn)
    return [db_dict[region],table_collection,row_collection]

In [44]:
db_collection_dict = {
    "main":get_collections_db("main")
}

In [45]:
text_to_sql = dspy.OpenAI(model='gpt-3.5-turbo', max_tokens=1024)
sql_to_answer = dspy.OpenAI(model='gpt-3.5-turbo',max_tokens=1024)

# DSPy signature for converting text to SQL query
class TextToSQLAnswer(dspy.Signature):
    """Convert natural language text to SQL using suitable schema(s) from multiple schema choices"""

    question:str = dspy.InputField(desc="natural language input which will be converted to SQL")
    relevant_table_schemas_rows:str = dspy.InputField(desc="Multiple possible tables which has table name and corresponding columns, along with relevant rows from the table (values in the same order as columns above)")
    sql:str = dspy.OutputField(desc="Generate syntactically correct sqlite query with correct column names using suitable tables(s) and its rows.\n Don't forget to add distinct.\n Please rename the returned columns into suitable names.\n DON'T OUTPUT anything else other than the sqlite query")

# DSPy signature for converting SQL query and question to natural language text
class SQLReturnToAnswer(dspy.Signature):
    """Answer the question using the rows from the SQL query"""
    question:str = dspy.InputField()
    sql:str = dspy.InputField(desc="sqlite query that generated the rows")
    relevant_rows:str = dspy.InputField(desc="relevant rows to answer the question")
    answer:str = dspy.OutputField(desc="answer to the question using relevant rows and the sql query")

# If there is an SQLError, then rectify the error by trying again
class SQLRectifier(dspy.Signature):
    """Correct the SQL query to resolve the error using the proper table names, columns and rows"""
    input_sql:str = dspy.InputField(desc="sqlite query that needs to be fixed")
    error_str: str = dspy.InputField(desc="error that needs to be resolved")
    relevant_table_schemas_rows:str = dspy.InputField(desc="Multiple possible tables which has table name and corresponding columns, along with relevant rows from the table (values in the same order as columns above)")
    sql:str = dspy.OutputField(desc="corrected sqlite query to resolve the error and remove and any invalid syntax in the query.\n Don't output anything else other than the sqlite query")

dspy.settings.configure(lm=text_to_sql)

# Filter out the SQL Query
def process_sql_str(sql_str:str):
    sql_str = sql_str.replace("```","")
    sql_str = sql_str.replace("sql","")
    sql_str = sql_str.strip()
    return sql_str

<p align="center">
  <img src="https://raw.githubusercontent.com/Athe-kunal/Text-to-SQL/main/Schema.png" alt="Sublime's custom image"/>
</p>

In [46]:
def get_table_results(table_collection_,question:str):
    # question_emb = emb_fn.embed_with_retries(question)[0]
    # Get the table results for the given question
    table_results = table_collection_.query(
        query_texts = question,
        n_results = 5
    )
    # print(table_results['documents'][0])
    return table_results

def get_row_results(row_collection_,question,table_name:str):
    # Get the row results for the given question
    row_results = row_collection_.query(
        query_texts = question,
        where = {"table_name":table_name},
        n_results = 5
    )
    print(row_results['documents'][0])
    return row_results

In [47]:
from typing import Any
import re

class TextToSQLQueryModule(dspy.Module):
    """Text to SQL to final module"""
    def __init__(self,region:str,use_cot:bool=True,max_retries:int=3):
        """Text to Answer init module

        Args:
            region (str): Region for which the module will be used.
            use_cot (bool, optional): Whether to use chain of thought for sql query generation. Defaults to True.
            max_retries (int, optional): Number of max retries for SQLError. Defaults to 3.
        """
        super().__init__()
        self.region = region
        db,table_collection,row_collection = db_collection_dict[region]
        # print(db,table_collection,row_collection)
        self.table_collection = table_collection
        self.use_cot = use_cot
        self.db = db
        self.row_collection = row_collection
        if self.use_cot == True:
            self.sqlAnswer = dspy.ChainOfThought(TextToSQLAnswer)
        else:
            self.sqlAnswer = dspy.Predict(TextToSQLAnswer)
        self.final_output = dspy.Predict(SQLReturnToAnswer)
        self.max_tries = max_retries
        # Initialize the sql rectifier with CoT reasoning
        self.sql_rectifier = dspy.ChainOfThought(SQLRectifier,rationale_type=dspy.OutputField(
            prefix="Reasoning: Let's think step by step in order to",
            desc="${produce the answer}. We ..."
        ))

    def __call__(self, *args: Any, **kwargs: Any) -> Any:
        return self.forward(*args, **kwargs)

    def forward(self,question):
        # Embed the question with embedding function
        question_emb = emb_fn([question])[0]
        # Retrieve the relevant tables from table schema and table summary
        docs = self.table_collection.query(
            query_embeddings = question_emb,
            n_results = 5
        )
        # docs = get_table_results(db_collection_dict[self.region][1],question)
        relevant_rows_schemas = ""

        existing_table_names = []

        for table_idx,metadata_name in enumerate(docs['metadatas'][0]):
            table_metadata = metadata_name['table_metadata']
            table_name = metadata_name['table_name']
            # If the table name is already in the list of existing table names, skip it
            # if table_name in existing_table_names:
            #     continue
            existing_table_names.append(table_name)
            # Retrieve the relevant rows from the current table
            rows = self.row_collection.query(
                query_embeddings = question_emb,
                n_results = 5,
                # where clause to filter the rows
                where = {"table_name":table_name}
            )
            # Retrieve the relevant table with the schema and summary
            relevant_rows_schemas += f'Table name: {table_name} \n'
            relevant_rows_schemas += "/* \n"
            for match in re.finditer("columns: ",table_metadata):
                cols_end = match.end()
            relevant_rows_schemas += "col : " + " | ".join(table_metadata[cols_end:].split(", ")) + "\n"
            for row_idx,row in enumerate(rows['metadatas'][0]):
                # Get the relevant rows from the current table
                # relevant_rows_schemas += f'\tRow {row_idx+1} from table {table_name}: {row["full_rows"]}\n'
                relevant_rows_schemas += f'row {row_idx+1} : {" | ".join(row["full_rows"].split(", "))}\n'
            relevant_rows_schemas += "*/" + '\n\n'
        print(colored(relevant_rows_schemas,"yellow"))
        # return
        sql_query = self.sqlAnswer(question=question,relevant_table_schemas_rows=relevant_rows_schemas)

        num_tries = 0
        print(sql_query)
        while num_tries <= self.max_tries:
            with self.db.connect() as conn:
                try:
                    # Try executing the sql query for the database
                    result = conn.execute(text(process_sql_str(sql_query.sql)))
                    num_tries = self.max_tries + 1
                except Exception as error:
                    # If there is an sql error, then try again with the sql rectifier
                    print(colored(str(error),'red'))
                    sql_query = self.sql_rectifier(input_sql=sql_query.sql,error_str=str(error),relevant_table_schemas_rows=relevant_rows_schemas)
                    print(colored(sql_query.rationale,'green'))
                    print()
                    print(colored(sql_query.sql,'green'))
                    # If all the num_retries are exhausted, then exit the program
                    num_tries += 1
                    if num_tries == self.max_tries+1:
                        return sql_query,error
        # With the retrieved rows from the database, then try to answer the question with dspy context
        with dspy.context(lm=sql_to_answer):
            row_str = ""
            key = tuple(result.keys())
            for row in result.fetchall():
                for r,k in zip(row,key):
                    row_str += f" {k} = {r},"
                row_str = row_str[:-1]
                row_str += "\n"
            print(f"Extracted rows: {row_str}")
            final_answer = self.final_output(question=question,sql=sql_query.sql,relevant_rows=row_str)
            return final_answer
tsql_ = TextToSQLQueryModule("main")
question = "Can you give me the status of invoice number 1687546526?"
sq = tsql_(question = question)

Table name: AP_INVOICES_ALL_processed 
/* 
col : INVOICE_ID (INTEGER) | INVOICE_NUM (INTEGER) | VENDOR_ID (INTEGER) | VENDOR_NAME (VARCHAR) | PO_HEADER_ID (INTEGER) | INVOICE_AMOUNT (INTEGER) | INVOICE_DATE (VARCHAR) | LAST_UPDATE_DATE (VARCHAR) | LAST_UPDATED_BY (VARCHAR) | SET_OF_BOOKS_ID (INTEGER) | INVOICE_CURRENCY_CODE (VARCHAR) | PAYMENT_CURRENCY_CODE (VARCHAR) | PAYMENT_CROSS_RATE (INTEGER) | VENDOR_SITE_ID (INTEGER) | AMOUNT_PAID (INTEGER) | DISCOUNT_AMOUNT_TAKEN (INTEGER) | SOURCE (VARCHAR) | INVOICE_TYPE_LOOKUP_CODE (VARCHAR) | DESCRIPTION (VARCHAR) | BATCH_ID (INTEGER) | AMOUNT_APPLICABLE_TO_DISCOUNT (INTEGER) | TAX_AMOUNT (INTEGER) | TERMS_ID (INTEGER) | TERMS_DATE (VARCHAR) | PAYMENT_METHOD_CODE (VARCHAR) | PAY_GROUP_LOOKUP_CODE (VARCHAR) | ACCTS_PAY_CODE_COMBINATION_ID (INTEGER) | PAYMENT_STATUS_FLAG (VARCHAR) | CREATION_DATE (VARCHAR) | LAST_UPDATE_LOGIN (INTEGER) | EXCLUSIVE_PAYMENT_FLAG (VARCHAR) | FREIGHT_AMOUNT (INTEGER) | DOC_CATEGORY_CODE (VARCHAR) | APPROVAL_STATU

In [48]:
print(sq)

Prediction(
    answer='| invoice_status |\n|----------------|\n| APPROVED       |\n\nAnswer: The status of invoice number 1687546526 is APPROVED.'
)


In [50]:
print(sq.answer)

| invoice_status |
|----------------|
| APPROVED       |

Answer: The status of invoice number 1687546526 is APPROVED.


In [51]:
# tsql = TextToSQLQueryModule("main")
sq = tsql_(question="What is the currency code for the invoice number 1687546526?")
print(sq.answer)

Table name: AP_INVOICES_ALL_processed 
/* 
col : INVOICE_ID (INTEGER) | INVOICE_NUM (INTEGER) | VENDOR_ID (INTEGER) | VENDOR_NAME (VARCHAR) | PO_HEADER_ID (INTEGER) | INVOICE_AMOUNT (INTEGER) | INVOICE_DATE (VARCHAR) | LAST_UPDATE_DATE (VARCHAR) | LAST_UPDATED_BY (VARCHAR) | SET_OF_BOOKS_ID (INTEGER) | INVOICE_CURRENCY_CODE (VARCHAR) | PAYMENT_CURRENCY_CODE (VARCHAR) | PAYMENT_CROSS_RATE (INTEGER) | VENDOR_SITE_ID (INTEGER) | AMOUNT_PAID (INTEGER) | DISCOUNT_AMOUNT_TAKEN (INTEGER) | SOURCE (VARCHAR) | INVOICE_TYPE_LOOKUP_CODE (VARCHAR) | DESCRIPTION (VARCHAR) | BATCH_ID (INTEGER) | AMOUNT_APPLICABLE_TO_DISCOUNT (INTEGER) | TAX_AMOUNT (INTEGER) | TERMS_ID (INTEGER) | TERMS_DATE (VARCHAR) | PAYMENT_METHOD_CODE (VARCHAR) | PAY_GROUP_LOOKUP_CODE (VARCHAR) | ACCTS_PAY_CODE_COMBINATION_ID (INTEGER) | PAYMENT_STATUS_FLAG (VARCHAR) | CREATION_DATE (VARCHAR) | LAST_UPDATE_LOGIN (INTEGER) | EXCLUSIVE_PAYMENT_FLAG (VARCHAR) | FREIGHT_AMOUNT (INTEGER) | DOC_CATEGORY_CODE (VARCHAR) | APPROVAL_STATU

In [52]:
sq = tsql_(question="Can you give me the vendor name, amount paid and source for the same?")
print(sq.answer)

Table name: AP_SUPPLIERS_processed 
/* 
col : VENDOR_ID (INTEGER) | LAST_UPDATE_DATE (VARCHAR) | LAST_UPDATED_BY (VARCHAR) | VENDOR_NAME (VARCHAR) | VENDOR_NAME_ALT (VARCHAR) | SEGMENT1 (VARCHAR) | SUMMARY_FLAG (VARCHAR) | ENABLED_FLAG (VARCHAR) | LAST_UPDATE_LOGIN (INTEGER) | CREATION_DATE (VARCHAR) | CREATED_BY (VARCHAR) | EMPLOYEE_ID (INTEGER) | VENDOR_TYPE_LOOKUP_CODE (VARCHAR) | ONE_TIME_FLAG (VARCHAR) | TERMS_ID (INTEGER) | ALWAYS_TAKE_DISC_FLAG (VARCHAR) | PAY_DATE_BASIS_LOOKUP_CODE (VARCHAR) | PAYMENT_PRIORITY (INTEGER) | INVOICE_CURRENCY_CODE (VARCHAR) | PAYMENT_CURRENCY_CODE (VARCHAR) | INVOICE_AMOUNT_LIMIT (INTEGER) | HOLD_ALL_PAYMENTS_FLAG (VARCHAR) | HOLD_FUTURE_PAYMENTS_FLAG (VARCHAR) | START_DATE_ACTIVE (VARCHAR) | END_DATE_ACTIVE (VARCHAR) | PAYMENT_METHOD_LOOKUP_CODE (VARCHAR) | BANK_ACCOUNT_NAME (VARCHAR) | BANK_ACCOUNT_NUM (INTEGER) | BANK_NUM (INTEGER) | BANK_ACCOUNT_TYPE (VARCHAR) | WOMEN_OWNED_FLAG (VARCHAR) | SMALL_BUSINESS_FLAG (VARCHAR) | HOLD_FLAG (VARCHAR) | 

In [53]:
sq = tsql_(question="What is the amount paid for the invoice number 1687546526?")
print(sq.answer)

Table name: AP_INVOICES_ALL_processed 
/* 
col : INVOICE_ID (INTEGER) | INVOICE_NUM (INTEGER) | VENDOR_ID (INTEGER) | VENDOR_NAME (VARCHAR) | PO_HEADER_ID (INTEGER) | INVOICE_AMOUNT (INTEGER) | INVOICE_DATE (VARCHAR) | LAST_UPDATE_DATE (VARCHAR) | LAST_UPDATED_BY (VARCHAR) | SET_OF_BOOKS_ID (INTEGER) | INVOICE_CURRENCY_CODE (VARCHAR) | PAYMENT_CURRENCY_CODE (VARCHAR) | PAYMENT_CROSS_RATE (INTEGER) | VENDOR_SITE_ID (INTEGER) | AMOUNT_PAID (INTEGER) | DISCOUNT_AMOUNT_TAKEN (INTEGER) | SOURCE (VARCHAR) | INVOICE_TYPE_LOOKUP_CODE (VARCHAR) | DESCRIPTION (VARCHAR) | BATCH_ID (INTEGER) | AMOUNT_APPLICABLE_TO_DISCOUNT (INTEGER) | TAX_AMOUNT (INTEGER) | TERMS_ID (INTEGER) | TERMS_DATE (VARCHAR) | PAYMENT_METHOD_CODE (VARCHAR) | PAY_GROUP_LOOKUP_CODE (VARCHAR) | ACCTS_PAY_CODE_COMBINATION_ID (INTEGER) | PAYMENT_STATUS_FLAG (VARCHAR) | CREATION_DATE (VARCHAR) | LAST_UPDATE_LOGIN (INTEGER) | EXCLUSIVE_PAYMENT_FLAG (VARCHAR) | FREIGHT_AMOUNT (INTEGER) | DOC_CATEGORY_CODE (VARCHAR) | APPROVAL_STATU

In [54]:
print(sq.answer)

| Invoice_Number | Amount_Paid |
|----------------|-------------|
| 1687546526     | 500         |

Answer: The amount paid for the invoice number 1687546526 is $500.


In [55]:
sq = tsql_(question="What is the PO number for the invoice number 9302587107?")
print(sq.answer)

Table name: PO_DISTRIBUTIONS_ALL_processed 
/* 
col : PO_DISTRIBUTION_ID (INTEGER) | LAST_UPDATE_DATE (VARCHAR) | LAST_UPDATED_BY (VARCHAR) | PO_HEADER_ID (INTEGER) | PO_LINE_ID (INTEGER) | LINE_LOCATION_ID (INTEGER) | SET_OF_BOOKS_ID (INTEGER) | CODE_COMBINATION_ID (INTEGER) | QUANTITY_ORDERED (INTEGER) | LAST_UPDATE_LOGIN (INTEGER) | CREATION_DATE (VARCHAR) | CREATED_BY (VARCHAR) | QUANTITY_DELIVERED (INTEGER) | QUANTITY_BILLED (INTEGER) | QUANTITY_CANCELLED (INTEGER) | RATE_DATE (VARCHAR) | RATE (INTEGER) | AMOUNT_BILLED (INTEGER) | ENCUMBERED_FLAG (VARCHAR) | GL_CANCELLED_DATE (VARCHAR) | DESTINATION_TYPE_CODE (VARCHAR) | DESTINATION_ORGANIZATION_ID (INTEGER) | DESTINATION_SUBINVENTORY (VARCHAR) | ATTRIBUTE14 (VARCHAR) | ATTRIBUTE15 (VARCHAR) | WIP_ENTITY_ID (INTEGER) | ACCRUAL_ACCOUNT_ID (INTEGER) | VARIANCE_ACCOUNT_ID (INTEGER) | PREVENT_ENCUMBRANCE_FLAG (VARCHAR) | DESTINATION_CONTEXT (VARCHAR) | DISTRIBUTION_NUM (INTEGER) | REQUEST_ID (INTEGER) | PROGRAM_APPLICATION_ID (INTEGER

In [56]:
print(sq.answer)

| PO_NUMBER |
|-----------|
| 123456    |

Answer: The PO number for the invoice number 9302587107 is 123456.


In [57]:
sq = tsql_(question="What is the invoice amount, payment status and invoice date for invoice number SC16616G-DEC-24?")
print(sq.answer)

Table name: AP_INVOICES_ALL_processed 
/* 
col : INVOICE_ID (INTEGER) | INVOICE_NUM (INTEGER) | VENDOR_ID (INTEGER) | VENDOR_NAME (VARCHAR) | PO_HEADER_ID (INTEGER) | INVOICE_AMOUNT (INTEGER) | INVOICE_DATE (VARCHAR) | LAST_UPDATE_DATE (VARCHAR) | LAST_UPDATED_BY (VARCHAR) | SET_OF_BOOKS_ID (INTEGER) | INVOICE_CURRENCY_CODE (VARCHAR) | PAYMENT_CURRENCY_CODE (VARCHAR) | PAYMENT_CROSS_RATE (INTEGER) | VENDOR_SITE_ID (INTEGER) | AMOUNT_PAID (INTEGER) | DISCOUNT_AMOUNT_TAKEN (INTEGER) | SOURCE (VARCHAR) | INVOICE_TYPE_LOOKUP_CODE (VARCHAR) | DESCRIPTION (VARCHAR) | BATCH_ID (INTEGER) | AMOUNT_APPLICABLE_TO_DISCOUNT (INTEGER) | TAX_AMOUNT (INTEGER) | TERMS_ID (INTEGER) | TERMS_DATE (VARCHAR) | PAYMENT_METHOD_CODE (VARCHAR) | PAY_GROUP_LOOKUP_CODE (VARCHAR) | ACCTS_PAY_CODE_COMBINATION_ID (INTEGER) | PAYMENT_STATUS_FLAG (VARCHAR) | CREATION_DATE (VARCHAR) | LAST_UPDATE_LOGIN (INTEGER) | EXCLUSIVE_PAYMENT_FLAG (VARCHAR) | FREIGHT_AMOUNT (INTEGER) | DOC_CATEGORY_CODE (VARCHAR) | APPROVAL_STATU

In [58]:
print(sq.answer)

| invoice_amount | payment_status | invoice_date |
|----------------|----------------|--------------|
| 500.00         | PAID           | 2024-12-24   |

Answer: The invoice amount for invoice number SC16616G-DEC-24 is $500.00, the payment status is PAID, and the invoice date is December 24, 2024.


In [59]:
sq = tsql_(question="What is the vendor id, payment currency code, total tax amount and payment status for invoice number 9016618879 ?")
print(sq.answer)

Table name: AP_INVOICES_ALL_processed 
/* 
col : INVOICE_ID (INTEGER) | INVOICE_NUM (INTEGER) | VENDOR_ID (INTEGER) | VENDOR_NAME (VARCHAR) | PO_HEADER_ID (INTEGER) | INVOICE_AMOUNT (INTEGER) | INVOICE_DATE (VARCHAR) | LAST_UPDATE_DATE (VARCHAR) | LAST_UPDATED_BY (VARCHAR) | SET_OF_BOOKS_ID (INTEGER) | INVOICE_CURRENCY_CODE (VARCHAR) | PAYMENT_CURRENCY_CODE (VARCHAR) | PAYMENT_CROSS_RATE (INTEGER) | VENDOR_SITE_ID (INTEGER) | AMOUNT_PAID (INTEGER) | DISCOUNT_AMOUNT_TAKEN (INTEGER) | SOURCE (VARCHAR) | INVOICE_TYPE_LOOKUP_CODE (VARCHAR) | DESCRIPTION (VARCHAR) | BATCH_ID (INTEGER) | AMOUNT_APPLICABLE_TO_DISCOUNT (INTEGER) | TAX_AMOUNT (INTEGER) | TERMS_ID (INTEGER) | TERMS_DATE (VARCHAR) | PAYMENT_METHOD_CODE (VARCHAR) | PAY_GROUP_LOOKUP_CODE (VARCHAR) | ACCTS_PAY_CODE_COMBINATION_ID (INTEGER) | PAYMENT_STATUS_FLAG (VARCHAR) | CREATION_DATE (VARCHAR) | LAST_UPDATE_LOGIN (INTEGER) | EXCLUSIVE_PAYMENT_FLAG (VARCHAR) | FREIGHT_AMOUNT (INTEGER) | DOC_CATEGORY_CODE (VARCHAR) | APPROVAL_STATU

In [60]:
print(sq.answer)

| vendor_id | payment_currency_code | total_tax_amount | payment_status |
|-----------|-----------------------|------------------|-----------------|
| 12345     | USD                   | 50.00            | PAID            |

Answer: The vendor id for invoice number 9016618879 is 12345, the payment currency code is USD, the total tax amount is 50.00, and the payment status is PAID.


In [61]:
sq = tsql_(question="What is the invoice date for invoice number 9016618879 ?")
print(sq.answer)

Table name: AP_INVOICES_ALL_processed 
/* 
col : INVOICE_ID (INTEGER) | INVOICE_NUM (INTEGER) | VENDOR_ID (INTEGER) | VENDOR_NAME (VARCHAR) | PO_HEADER_ID (INTEGER) | INVOICE_AMOUNT (INTEGER) | INVOICE_DATE (VARCHAR) | LAST_UPDATE_DATE (VARCHAR) | LAST_UPDATED_BY (VARCHAR) | SET_OF_BOOKS_ID (INTEGER) | INVOICE_CURRENCY_CODE (VARCHAR) | PAYMENT_CURRENCY_CODE (VARCHAR) | PAYMENT_CROSS_RATE (INTEGER) | VENDOR_SITE_ID (INTEGER) | AMOUNT_PAID (INTEGER) | DISCOUNT_AMOUNT_TAKEN (INTEGER) | SOURCE (VARCHAR) | INVOICE_TYPE_LOOKUP_CODE (VARCHAR) | DESCRIPTION (VARCHAR) | BATCH_ID (INTEGER) | AMOUNT_APPLICABLE_TO_DISCOUNT (INTEGER) | TAX_AMOUNT (INTEGER) | TERMS_ID (INTEGER) | TERMS_DATE (VARCHAR) | PAYMENT_METHOD_CODE (VARCHAR) | PAY_GROUP_LOOKUP_CODE (VARCHAR) | ACCTS_PAY_CODE_COMBINATION_ID (INTEGER) | PAYMENT_STATUS_FLAG (VARCHAR) | CREATION_DATE (VARCHAR) | LAST_UPDATE_LOGIN (INTEGER) | EXCLUSIVE_PAYMENT_FLAG (VARCHAR) | FREIGHT_AMOUNT (INTEGER) | DOC_CATEGORY_CODE (VARCHAR) | APPROVAL_STATU

In [62]:
print(sq.answer)

| Invoice_Date |
|--------------|
| 2022-01-15   |

Answer: The invoice date for invoice number 9016618879 is 2022-01-15.


In [85]:
sq = tsql_(question="Can you give the payment status for above too?")
print(sq.answer)

Table name: AP_CHECKS_ALL_processed 
/* 
col : AMOUNT (INTEGER) | BANK_ACCOUNT_ID (INTEGER) | BANK_ACCOUNT_NAME (VARCHAR) | CHECK_DATE (VARCHAR) | CHECK_ID (INTEGER) | CHECK_NUMBER (INTEGER) | CURRENCY_CODE (VARCHAR) | LAST_UPDATED_BY (VARCHAR) | LAST_UPDATE_DATE (VARCHAR) | PAYMENT_METHOD_LOOKUP_CODE (VARCHAR) | PAYMENT_TYPE_FLAG (VARCHAR) | ADDRESS_LINE1 (VARCHAR) | CHECKRUN_NAME (VARCHAR) | CITY (VARCHAR) | COUNTRY (VARCHAR) | CREATED_BY (VARCHAR) | CREATION_DATE (VARCHAR) | STATUS_LOOKUP_CODE (VARCHAR) | VENDOR_NAME (VARCHAR) | ZIP (INTEGER) | BANK_ACCOUNT_NUM (INTEGER) | BANK_ACCOUNT_TYPE (VARCHAR) | BANK_NUM (INTEGER) | CHECK_VOUCHER_NUM (INTEGER) | CLEARED_AMOUNT (INTEGER) | CLEARED_DATE (VARCHAR) | ORG_ID (INTEGER) | VENDOR_ID (INTEGER) | VENDOR_SITE_ID (INTEGER) | EXCHANGE_DATE (VARCHAR) | CHECKRUN_ID (INTEGER) | REQUEST_ID (INTEGER) | POSITIVE_PAY_STATUS_CODE (VARCHAR) | ANTICIPATED_VALUE_DATE (VARCHAR) | CE_BANK_ACCT_USE_ID (INTEGER) | PAYMENT_METHOD_CODE (VARCHAR) | PARTY_I

In [82]:
import shutil
from google.colab import files
folder_to_zip = "./main_TABLE"
out = "main_TABLE.zip"
shutil.make_archive("main_TABLE", 'zip', folder_to_zip)
files.download(out)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>